# Muon vs. AdamW Experiment Analysis

This notebook analyzes the results of the OMat24 experiments, comparing the performance and learning geometry of the Muon optimizer against AdamW.


In [ ]:
%pip install pandas matplotlib seaborn


: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the metrics data
try:
    df = pd.read_csv('results/metrics.csv')
    # The first run has a generic name, let's rename it for clarity
    df['run'] = df['run'].replace('1', 'omat24_baseline')
    print("Data loaded successfully.")
    display(df.head())
except FileNotFoundError:
    print("Error: results/metrics.csv not found.")
    print("Please make sure you have run the experiments first.")

# Set plot style
sns.set_theme(style="whitegrid")


In [ ]:
plt.figure(figsize=(12, 8))
ax = sns.lineplot(data=df, x='epoch', y='val_mae', hue='run', marker='o')

ax.set_title('Validation MAE vs. Epoch for Different Optimizers', fontsize=16)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Validation Mean Absolute Error (MAE)', fontsize=12)
ax.legend(title='Optimizer Run')

# Annotate the minimum MAE for each run
for run_name in df['run'].unique():
    run_df = df[df['run'] == run_name]
    min_mae_epoch = run_df.loc[run_df['val_mae'].idxmin()]
    
    plt.annotate(f'{min_mae_epoch["val_mae"]:.4f}',
                 (min_mae_epoch['epoch'], min_mae_epoch['val_mae']),
                 textcoords="offset points",
                 xytext=(0,10),
                 ha='center',
                 arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2"))

plt.tight_layout()
plt.show()
